<a href="https://colab.research.google.com/github/Mozzer2310/text-mining-cwk/blob/sam-experiments/DL_Relation_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get the Dataset
We need to install the `datasets` module to download the [DialogRE](https://huggingface.co/datasets/dataset-org/dialog_re) dataset.

In [1]:
! pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00


Then we can download the dataset.

In [2]:
from datasets import load_dataset

dataset = load_dataset("dataset-org/dialog_re", download_mode="force_redownload", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

dialog_re.py:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

dialog_re.py:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1073 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/358 [00:00<?, ? examples/s]

Then view the Dataset and its contents.

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'relation_data'],
        num_rows: 1073
    })
    test: Dataset({
        features: ['dialog', 'relation_data'],
        num_rows: 357
    })
    validation: Dataset({
        features: ['dialog', 'relation_data'],
        num_rows: 358
    })
})

In [4]:
dataset['train'][0]

{'dialog': ["Speaker 1: It's been an hour and not one of my classmates has shown up! I tell you, when I actually die some people are gonna get seriously haunted!",
  'Speaker 2: There you go! Someone came!',
  "Speaker 1: Ok, ok! I'm gonna go hide! Oh, this is so exciting, my first mourner!",
  'Speaker 3: Hi, glad you could come.',
  'Speaker 2: Please, come in.',
  "Speaker 4: Hi, you're Chandler Bing, right? I'm Tom Gordon, I was in your class.",
  'Speaker 2: Oh yes, yes... let me... take your coat.',
  "Speaker 4: Thanks... uh... I'm so sorry about Ross, it's...",
  'Speaker 2: At least he died doing what he loved... watching blimps.',
  'Speaker 1: Who is he?',
  'Speaker 2: Some guy, Tom Gordon.',
  "Speaker 1: I don't remember him, but then again I touched so many lives.",
  'Speaker 3: So, did you know Ross well?',
  "Speaker 4: Oh, actually I barely knew him. Yeah, I came because I heard Chandler's news. D'you know if he's seeing anyone?",
  'Speaker 3: Yes, he is. Me.',
  'S

## Defining Constants
Constant definitions for the model.

In [5]:
# MODEL_NAME = "FacebookAI/roberta-base"
MODEL_NAME = "bert-base-uncased"
SEQ_LENGTH = 512

## Preprocess the Data
1. Reformat the dataset so each sample (relation) is extracted from each item in the dataset
2. Proprocess each sample getting the tokens and the positional indices of the entities
3. Create a PyTorch dataset for the data

### Reformat the Dataset
Convert the dataset so that each item contains a singular relation.

In [6]:
def reformat_dataset(dataset, add_triggers=True):
    reformatted_dataset = []

    for item in dataset:
        dialog = item['dialog']
        relation_data = item['relation_data']

        # Join the dialog into a single string
        all_dialog = ' '.join(dialog)

        samples = []
        for x, y, r, t in zip(relation_data['x'], relation_data['y'], relation_data['r'], relation_data['t']):
            sample = {'dialog': all_dialog, 'x': x, 'y': y, 'relation': r}
            if add_triggers:
                sample['trigger'] = t
            samples.append(sample)

        reformatted_dataset.extend(samples)

    return reformatted_dataset

In [7]:
reformatted_dataset = {}
for split in dataset.keys():
    reformatted_dataset[split] = reformat_dataset(dataset[split], add_triggers=False)

In [8]:
print(reformatted_dataset['train'][0])

{'dialog': "Speaker 1: It's been an hour and not one of my classmates has shown up! I tell you, when I actually die some people are gonna get seriously haunted! Speaker 2: There you go! Someone came! Speaker 1: Ok, ok! I'm gonna go hide! Oh, this is so exciting, my first mourner! Speaker 3: Hi, glad you could come. Speaker 2: Please, come in. Speaker 4: Hi, you're Chandler Bing, right? I'm Tom Gordon, I was in your class. Speaker 2: Oh yes, yes... let me... take your coat. Speaker 4: Thanks... uh... I'm so sorry about Ross, it's... Speaker 2: At least he died doing what he loved... watching blimps. Speaker 1: Who is he? Speaker 2: Some guy, Tom Gordon. Speaker 1: I don't remember him, but then again I touched so many lives. Speaker 3: So, did you know Ross well? Speaker 4: Oh, actually I barely knew him. Yeah, I came because I heard Chandler's news. D'you know if he's seeing anyone? Speaker 3: Yes, he is. Me. Speaker 4: What? You... You... Oh! Can I ask you a personal question? Ho-how 

### Preprocess each Sample

In [9]:
import re
import torch
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding="max_length", max_length=SEQ_LENGTH)
SUBJ_TOKEN = '[SUBJ]'
OBJ_TOKEN = '[OBJ]'
special_tokens_dict = {'additional_special_tokens': [SUBJ_TOKEN, OBJ_TOKEN]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def truncate_dialog_with_entities(tokenizer, text):
    """
    Truncates the dialog text while ensuring that the subject and object tokens are included.

    Args:
        tokenizer: The tokenizer object.
        text: The full text containing the dialog and entities.

    Returns:
         A tuple containing:
          - The truncated text.
          - List of tokens in the truncated text
    """

    parts = text.split('[SEP]')
    dialog = parts[0].strip()
    entities_and_triggers = '[SEP]' + '[SEP]'.join(parts[1:])

    tokens_full = tokenizer.tokenize(dialog)
    tokens_entities_and_triggers = tokenizer.tokenize(entities_and_triggers)

    e1_positions_full = [i for i, token in enumerate(tokens_full) if token == SUBJ_TOKEN]
    e2_positions_full = [i for i, token in enumerate(tokens_full) if token == OBJ_TOKEN]

    if len(tokens_full) + len(tokens_entities_and_triggers) <= SEQ_LENGTH:
        return text, tokenizer.tokenize(text)  # No truncation needed

    first_entity_index = min(e1_positions_full + e2_positions_full) if (e1_positions_full and e2_positions_full) else 0
    last_entity_index = max(e1_positions_full + e2_positions_full) if (e1_positions_full and e2_positions_full) else len(tokens_full)

    available_length_for_dialog = SEQ_LENGTH - len(tokens_entities_and_triggers)

    start_index = max(0, first_entity_index - (available_length_for_dialog // 4))
    end_index = min(len(tokens_full), last_entity_index + (available_length_for_dialog // 4))

    if end_index - start_index > available_length_for_dialog:
        end_index = min(len(tokens_full), start_index + available_length_for_dialog)

    if end_index - start_index > available_length_for_dialog:
        start_index = max(0, end_index - available_length_for_dialog)


    truncated_tokens_dialog = tokens_full[start_index:end_index]
    truncated_dialog = tokenizer.convert_tokens_to_string(truncated_tokens_dialog)

    truncated_text = f"{truncated_dialog} {entities_and_triggers.strip()}"

    return truncated_text, tokenizer.tokenize(truncated_text)

In [12]:
def preprocess_sample(sample, max_positions=20):
    SEP_TOKEN = '[SEP]'
    dialog = sample['dialog']
    x = sample['x']
    y = sample['y']

    dialog1 = dialog.replace(x, '[SUBJ]')
    dialog2 = dialog1.replace(y, '[OBJ]')

    text = f"{dialog2} {SEP_TOKEN} {x} {SEP_TOKEN} {y}"
    if 'trigger' in sample:
        trigger = ', '.join(sample['trigger'])
        text += f" {SEP_TOKEN} {trigger}"

    truncated_text, _ = truncate_dialog_with_entities(tokenizer, text)

    # Tokenize the (potentially truncated) text
    tokens = tokenizer(truncated_text, padding="max_length", truncation=True, max_length=SEQ_LENGTH, return_tensors="pt")

    # Find entity positions within the (potentially truncated) text
    words = tokenizer.convert_ids_to_tokens(tokens.input_ids.squeeze())
    e1_positions = [i for i, x in enumerate(words) if x == SUBJ_TOKEN]
    e2_positions = [i for i, x in enumerate(words) if x == OBJ_TOKEN]

    # Pad with -1 to a fixed length
    e1_positions += [-1] * (max_positions - len(e1_positions))
    e2_positions += [-1] * (max_positions - len(e2_positions))

    return {
        'tokens': tokens,
        'e1_positions': torch.tensor(e1_positions),
        'e2_positions': torch.tensor(e2_positions)
    }

In [13]:
preprocess_sample(reformatted_dataset['train'][0])

{'tokens': {'input_ids': tensor([[  101,  5882,  1015,  1024,  2009,  1005,  1055,  2042,  2019,  3178,
           1998,  2025,  2028,  1997,  2026, 19846,  2038,  3491,  2039,   999,
           1045,  2425,  2017,  1010,  2043,  1045,  2941,  3280,  2070,  2111,
           2024,  6069,  2131,  5667, 11171,   999, 30522,  1024,  2045,  2017,
           2175,   999,  2619,  2234,   999,  5882,  1015,  1024,  7929,  1010,
           7929,   999,  1045,  1005,  1049,  6069,  2175,  5342,   999,  2821,
           1010,  2023,  2003,  2061, 10990,  1010,  2026,  2034,  9587, 21737,
           2099,   999,  5882,  1017,  1024,  7632,  1010,  5580,  2017,  2071,
           2272,  1012, 30522,  1024,  3531,  1010,  2272,  1999,  1012,  5882,
           1018,  1024,  7632,  1010,  2017,  1005,  2128, 30523,  1010,  2157,
           1029,  1045,  1005,  1049,  3419,  5146,  1010,  1045,  2001,  1999,
           2115,  2465,  1012, 30522,  1024,  2821,  2748,  1010,  2748,  1012,
           1012,

### Convert the Relation Label to Multi-hot Vector

Function to get all the relation labels

In [14]:
def get_labels(dataset):
    all_dataset_labels = set()
    for datapoint in dataset['train']:
        for relation in  [item for sublist in datapoint['relation_data']['r'] for item in sublist]:
            all_dataset_labels.add(relation)
    return list(all_dataset_labels)

## Create PyTorch Dataset

In [15]:
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MultiLabelBinarizer

In [16]:
# Initialize the binarizer
class_labels = get_labels(dataset)
mlb = MultiLabelBinarizer(classes=class_labels)
# Fit the binarizer to the training labels
mlb.fit([item['relation'] for item in reformatted_dataset['train']])

MultiLabelBinarizer(classes=['per:children', 'per:roommate', 'per:age',
                             'per:major', 'per:employee_or_member_of',
                             'per:parents', 'per:place_of_residence',
                             'per:acquaintance', 'per:negative_impression',
                             'per:alternate_names', 'per:place_of_work',
                             'per:schools_attended', 'per:works', 'per:alumni',
                             'per:positive_impression', 'per:date_of_birth',
                             'gpe:visitors_of_place', 'per:spouse',
                             'org:employees_or_members', 'per:girl/boyfriend',
                             'per:boss', 'per:visited_place', 'per:neighbor',
                             'per:origin', 'gpe:residents_of_place',
                             'unanswerable', 'per:other_family', 'per:pet',
                             'per:title', 'per:siblings', ...])

In [17]:
# Create PyTorch Dataset
class DialogREDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        processed_sample = preprocess_sample(item)
        label = torch.tensor(mlb.fit_transform([item['relation']])[0], dtype=torch.float32)
        return {
            'input_ids': processed_sample['tokens'].input_ids.squeeze(0),
            'attention_mask': processed_sample['tokens'].attention_mask.squeeze(0),
            'e1_positions': processed_sample['e1_positions'],
            'e2_positions': processed_sample['e2_positions'],
            'labels': label
        }

In [18]:
train_dataset = DialogREDataset(reformatted_dataset['train'], tokenizer)
test_dataset = DialogREDataset(reformatted_dataset['test'], tokenizer)
validation_dataset = DialogREDataset(reformatted_dataset['validation'], tokenizer)

In [19]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig, BertModel # Using BertModel to fetch the base model without the extra head

In [20]:
class MultiLabelBERT(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        if model_name == "FacebookAI/roberta-base":
            config = RobertaConfig.from_pretrained("roberta-base", max_position_embeddings=SEQ_LENGTH)
            self.bert = RobertaModel(config)
        else:
            self.bert = BertModel.from_pretrained(model_name)

        input_hidden_size = self.bert.config.hidden_size

        # Resize the token embeddings to accommodate the new tokens
        self.bert.resize_token_embeddings(len(tokenizer))

        # Add our own classification head for predicting multiple labels
        self.classifier = nn.Sequential(
            nn.Linear(input_hidden_size * 2, SEQ_LENGTH), # Multiply input size by 2 as we have two entities
            nn.ReLU(),
            nn.Linear(SEQ_LENGTH, len(class_labels)), # There are 35 relations in the dataset
            nn.Sigmoid() # Sigmoid activation is used to output probabilities for each label
        )

    def avg_embedding(self, bert_output, entity_positions):
        # Filter out zero positions
        entity_positions_filtered = entity_positions[entity_positions != 0]

        #
        if len(entity_positions_filtered) == 0:
            return torch.zeros((bert_output.shape[-1]), dtype=torch.float32).to(bert_output.device)

        entity_embs = bert_output[:, entity_positions_filtered, :]
        return torch.mean(entity_embs, dim=1)  # Mean over all occurrences

    def forward(self, input_ids, attention_mask, e1_positions, e2_positions):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

        # Extract entity representations and apply averaging for multiple occurences
        e1_embedding = self.avg_embedding(bert_output, e1_positions)
        e2_embedding = self.avg_embedding(bert_output, e2_positions)

        # Concatenate entity embeddings
        combined_embedding = torch.cat([e1_embedding, e2_embedding], dim=-1)

        # Feed into the neural network classifier
        logits = self.classifier(combined_embedding)

        # Return relation probabilities for each label
        return logits

In [80]:
from transformers import Trainer, TrainingArguments
from google.colab import drive
import numpy as np

drive.mount('/content/drive')
OUTPUT_DIR = '/content/drive/MyDrive/re_models/'
LOGS_DIR = '/content/drive/MyDrive/re_models/logs'

training_args = TrainingArguments(
    optim="adamw_torch",
    output_dir='OUTPUT_DIR',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=LOGS_DIR,            # directory for storing logs
    logging_steps=100,
    #save_steps=100,
    learning_rate=2e-5,
    eval_steps=100
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

model = MultiLabelBERT(MODEL_NAME)
trainer.train()
trainer.save_model(OUTPUT_DIR)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-80-e77b97d29bce>, line 19)

In [21]:
# from torch.utils.data import DataLoader
# from transformers import AdamW, get_linear_schedule_with_warmup
# from tqdm import tqdm

# # Create dataset and model
# model = MultiLabelBERT(MODEL_NAME)

# # Create DataLoader
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # Training loop
# num_epochs = 3

# # Set up optimizer, scheduler and loss
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# total_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
# loss_fn = nn.BCELoss()  # Binary cross-entropy loss is used for multi-label problems

# # Use CUDA
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0
#     progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)

#     for batch in progress_bar:
#         # Load batches onto device
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         e1_positions = batch['e1_positions'].to(device)
#         e2_positions = batch['e2_positions'].to(device)
#         labels = batch['labels'].to(device)

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         logits = model(input_ids, attention_mask, e1_positions, e2_positions)

#         # Compute loss
#         loss = loss_fn(logits, labels)
#         total_loss += loss.item()

#         # Backward pass
#         loss.backward()

#         # Update weights
#         optimizer.step()
#         scheduler.step()

#         # Update progress bar
#         progress_bar.set_postfix({'loss': loss.item()})

#     avg_loss = total_loss / len(train_dataloader)
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss}')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Epoch 1/3:   1%|          | 4/750 [00:04<11:22,  1.09it/s, loss=0.668]Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


Epoch 1/3, Loss: 0.13731317176421484


Epoch 2/3, Loss: 0.0883039229263862


Epoch 3/3, Loss: 0.07745193116615216


# Inferencing

In [79]:
def predict_relations(sample_data, model, tokenizer, threshold=0.5):
  processed_sample = preprocess_sample(sample_data)
  input_ids = processed_sample['tokens'].input_ids.to(device)
  attention_mask = processed_sample['tokens'].attention_mask.to(device)
  e1_positions = processed_sample['e1_positions'].to(device)
  e2_positions = processed_sample['e2_positions'].to(device)

  with torch.no_grad():
    relation_probabilities = model(input_ids, attention_mask, e1_positions, e2_positions)

  # Apply threshold to get predicted relations
  relation_indices = np.where(relation_probabilities.cpu().numpy() > threshold)[1]

  return relation_indices  # List of relation indices

sample_data = reformatted_dataset['test'][501]
for key in sample_data:
  print(key, sample_data[key])


sample_label = torch.tensor(mlb.fit_transform([sample_data['relation']])[0], dtype=torch.float32)
print(np.where(sample_label.numpy() == 1))
relation_indices = predict_relations(sample_data, model, tokenizer, threshold=0.2)
predicted_relations = [class_labels[i] for i in relation_indices]
print(predicted_relations)

dialog Speaker 1: Brrrrrrr! Speaker 2: Hell is filled with people like you. Speaker 3: He's back! The peeper's back! Speaker 3: Get down! Speaker 4: Get down? Speaker 2: ...And boogie! Speaker 4: Thanks, but I gotta go to work and get my eyes scratched out by Mindy. Speaker 1: Relax. Y'know, she may not even know. Speaker 4: Please. I haven't heard from her in seven months, and now she calls me? I mean, what else is it about? Oh! She was my best friend, you guys! We went to camp together... she taught me how to kiss.. Speaker 3: Yeah? Speaker 4: And now, y'know, I'm like... I'm like the other woman! I feel so.. Speaker 3: ..Naughty! Speaker 4: Right, I'll see you guys later... Speaker 3: Oh, hold up, I'll walk out with you. Now, Rach, when she taught you to kiss, you were at camp, and.. were you wearing any kinda little uniform, or- That's fine, yeah…
x Speaker 4
y Rach
relation ['per:alternate_names']
(array([9]),)
['per:alternate_names']
